In [1]:
import time, pickle
import numpy as np
import cv2
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
import matplotlib.pyplot as plt
from utils import *

import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

In [2]:
n_of_trials = 500
batch_size = 1

epsilon_list=[+16/255, -16/255]


random_seed = 1
torch.backends.cudnn.enabled = True
torch.manual_seed(random_seed)

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [4]:
net = torch.load("net.pth")
net = net.to(device)

In [5]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


testset = torchvision.datasets.CIFAR10(root='./cifar10/', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified


In [6]:
"""
mode:
    0 -> uniform
    1 -> gaussian
    2 -> saliency
"""

mode = 0
max_steps = len(testset)

timer = Timer(max_steps)
experiment = Experiment(max_steps, np.array([32, 32]))


for i, (attack_image, attack_label) in enumerate(testloader, 0):
    if i==max_steps:
        break
    timer.start()
    record = Record(i, np.array([32, 32]))
    record.orig_input = attack_image

    
    with torch.no_grad():
        net.eval()
        outputs = net(attack_image.to(device))
        outputs = F.softmax(outputs, dim=1)

        confidence, predicted = torch.max(outputs.data, 1)
        
        record.gt = attack_label.item()
        record.init_pred = predicted.item()
        record.init_conf = confidence.item()
        
        
        print("---")
        if predicted.item() != attack_label.item():
            record.statue = 0 # wrong pred
            timer.end()
            experiment.add_record(record)

            print(record)
            print(timer)

            continue
        
    #SIMBA
    
    last_confidence = confidence

    cumulative_gradient = torch.zeros_like(attack_image)

    for n in range(n_of_trials):
        with torch.no_grad():
            net.eval()
            
            if mode == 0:
                # uniform sampling
                index = torch.randint(attack_image.shape[2], (2, ))
            elif mode == 1:
                # standard gaussian sampling
                index = (torch.normal(0,1, (2,))*5+16).to(torch.long)
                for i, ind in enumerate(index):
                    if ind<0:
                        index[i]=0
                    if ind>attack_image.shape[2]-1:
                        index[i]=attack_image.shape[2]-1
            elif mode == 2:
                # saliency map sampling
                saliency_map = create_saliency_map(attack_image[0].permute(1, 2, 0).detach().numpy())
                row, column = sample_saliency(saliency_map/255)
                index = [row, column]
            #print(index)

            for epsilon in epsilon_list:

                attack_image_test = torch.add(attack_image, cumulative_gradient)
                
                attack_image_test[:,:, index[0], index[1]]+=epsilon


                outputs = net(attack_image_test.to(device))
                outputs = F.softmax(outputs, dim=1)

                confidence, predicted = torch.max(outputs.data, 1)
                confidence = confidence.item()
                predicted = predicted.item()
                
                if confidence < last_confidence:
                    cumulative_gradient[:,:, index[0], index[1]]+=epsilon
                    last_confidence= confidence
                    break
                    
            if predicted!=attack_label.item(): #network is fooled
                record.last_pred = predicted
                record.last_conf = confidence
                record.noise = cumulative_gradient[0]
                record.n_queries = n
                record.statue = 1 # successful atack
                
                experiment.add_record(record)
                
                timer.end()
                
                print(record)
                print(timer)

                break
                
            if n == (n_of_trials-1):
                record.last_pred = predicted
                record.last_conf = confidence
                record.noise = cumulative_gradient[0]
                record.n_queries = n
                record.statue = 2 # unsuccessful atack
                
                experiment.add_record(record)
                
                timer.end()
                
                print(record)
                print(timer)
                
        
    
save_pickle("experiment_{}".format(mode), experiment)    
        

---
ID: 0 | Attack Unsuccessful!| GT: 3 | Init. Pred: 3 | Init. Conf: 1.00 | Last Pred: 3 | Last Conf: 1.00 | Query: 499
Remaining Time: 1794.11 mins
---
ID: 1 | Attack Successful!| GT: 8 | Init. Pred: 8 | Init. Conf: 0.82 | Last Pred: 1 | Last Conf: 0.57 | Query: 11
Remaining Time: 913.02 mins
---
ID: 2 | Wrong Prediction! | GT: 8 | Pred: 1 | Conf: 1.00
Remaining Time: 609.17 mins
---
ID: 3 | Attack Successful!| GT: 0 | Init. Pred: 0 | Init. Conf: 1.00 | Last Pred: 8 | Last Conf: 0.51 | Query: 150
Remaining Time: 554.93 mins
---
ID: 4 | Attack Successful!| GT: 6 | Init. Pred: 6 | Init. Conf: 1.00 | Last Pred: 2 | Last Conf: 0.61 | Query: 107
Remaining Time: 498.14 mins
---
ID: 5 | Attack Successful!| GT: 6 | Init. Pred: 6 | Init. Conf: 1.00 | Last Pred: 3 | Last Conf: 0.51 | Query: 133
Remaining Time: 473.55 mins
---
ID: 6 | Attack Successful!| GT: 1 | Init. Pred: 1 | Init. Conf: 1.00 | Last Pred: 9 | Last Conf: 0.52 | Query: 163
Remaining Time: 462.30 mins
---
ID: 7 | Attack Unsucces

ID: 60 | Attack Unsuccessful!| GT: 7 | Init. Pred: 7 | Init. Conf: 1.00 | Last Pred: 7 | Last Conf: 1.00 | Query: 499
Remaining Time: 809.71 mins
---
ID: 61 | Attack Successful!| GT: 3 | Init. Pred: 3 | Init. Conf: 0.51 | Last Pred: 5 | Last Conf: 0.53 | Query: 1
Remaining Time: 796.70 mins
---
ID: 62 | Attack Unsuccessful!| GT: 6 | Init. Pred: 6 | Init. Conf: 1.00 | Last Pred: 6 | Last Conf: 1.00 | Query: 499
Remaining Time: 809.58 mins
---
ID: 63 | Attack Unsuccessful!| GT: 3 | Init. Pred: 3 | Init. Conf: 1.00 | Last Pred: 3 | Last Conf: 1.00 | Query: 499
Remaining Time: 822.26 mins
---
ID: 64 | Attack Successful!| GT: 6 | Init. Pred: 6 | Init. Conf: 1.00 | Last Pred: 3 | Last Conf: 0.58 | Query: 203
Remaining Time: 818.01 mins
---
ID: 65 | Attack Successful!| GT: 2 | Init. Pred: 2 | Init. Conf: 1.00 | Last Pred: 6 | Last Conf: 0.53 | Query: 90
Remaining Time: 808.81 mins
---
ID: 66 | Attack Successful!| GT: 1 | Init. Pred: 1 | Init. Conf: 1.00 | Last Pred: 8 | Last Conf: 0.54 | Quer

ID: 117 | Attack Successful!| GT: 4 | Init. Pred: 4 | Init. Conf: 1.00 | Last Pred: 5 | Last Conf: 0.50 | Query: 99
Remaining Time: 813.02 mins
---
ID: 118 | Wrong Prediction! | GT: 2 | Pred: 3 | Conf: 0.47
Remaining Time: 806.12 mins
---
ID: 119 | Attack Successful!| GT: 7 | Init. Pred: 7 | Init. Conf: 1.00 | Last Pred: 5 | Last Conf: 0.51 | Query: 48
Remaining Time: 800.32 mins
---
ID: 120 | Attack Successful!| GT: 8 | Init. Pred: 8 | Init. Conf: 1.00 | Last Pred: 1 | Last Conf: 0.50 | Query: 53
Remaining Time: 794.77 mins
---
ID: 121 | Attack Successful!| GT: 3 | Init. Pred: 3 | Init. Conf: 1.00 | Last Pred: 0 | Last Conf: 0.48 | Query: 37
Remaining Time: 788.83 mins
---
ID: 122 | Attack Unsuccessful!| GT: 1 | Init. Pred: 1 | Init. Conf: 1.00 | Last Pred: 1 | Last Conf: 1.00 | Query: 499
Remaining Time: 795.48 mins
---
ID: 123 | Attack Unsuccessful!| GT: 2 | Init. Pred: 2 | Init. Conf: 1.00 | Last Pred: 2 | Last Conf: 1.00 | Query: 499
Remaining Time: 801.92 mins
---
ID: 124 | Attac

ID: 176 | Attack Unsuccessful!| GT: 3 | Init. Pred: 3 | Init. Conf: 1.00 | Last Pred: 3 | Last Conf: 1.00 | Query: 499
Remaining Time: 775.06 mins
---
ID: 177 | Attack Unsuccessful!| GT: 7 | Init. Pred: 7 | Init. Conf: 1.00 | Last Pred: 7 | Last Conf: 1.00 | Query: 499
Remaining Time: 779.57 mins
---
ID: 178 | Attack Successful!| GT: 5 | Init. Pred: 5 | Init. Conf: 1.00 | Last Pred: 3 | Last Conf: 0.70 | Query: 65
Remaining Time: 776.06 mins
---
ID: 179 | Attack Successful!| GT: 0 | Init. Pred: 0 | Init. Conf: 1.00 | Last Pred: 9 | Last Conf: 0.51 | Query: 150
Remaining Time: 773.80 mins
---
ID: 180 | Attack Unsuccessful!| GT: 0 | Init. Pred: 0 | Init. Conf: 1.00 | Last Pred: 0 | Last Conf: 1.00 | Query: 499
Remaining Time: 778.14 mins
---
ID: 181 | Attack Successful!| GT: 5 | Init. Pred: 5 | Init. Conf: 1.00 | Last Pred: 7 | Last Conf: 0.51 | Query: 279
Remaining Time: 777.57 mins
---
ID: 182 | Attack Unsuccessful!| GT: 2 | Init. Pred: 2 | Init. Conf: 1.00 | Last Pred: 2 | Last Conf: 

ID: 235 | Attack Unsuccessful!| GT: 0 | Init. Pred: 0 | Init. Conf: 1.00 | Last Pred: 0 | Last Conf: 1.00 | Query: 499
Remaining Time: 744.65 mins
---
ID: 236 | Attack Unsuccessful!| GT: 4 | Init. Pred: 4 | Init. Conf: 1.00 | Last Pred: 4 | Last Conf: 1.00 | Query: 499
Remaining Time: 748.13 mins
---
ID: 237 | Wrong Prediction! | GT: 0 | Pred: 9 | Conf: 0.80
Remaining Time: 744.91 mins
---
ID: 238 | Attack Successful!| GT: 5 | Init. Pred: 5 | Init. Conf: 1.00 | Last Pred: 3 | Last Conf: 0.55 | Query: 110
Remaining Time: 742.82 mins
---
ID: 239 | Attack Unsuccessful!| GT: 5 | Init. Pred: 5 | Init. Conf: 1.00 | Last Pred: 5 | Last Conf: 1.00 | Query: 499
Remaining Time: 746.03 mins
---
ID: 240 | Attack Unsuccessful!| GT: 1 | Init. Pred: 1 | Init. Conf: 1.00 | Last Pred: 1 | Last Conf: 1.00 | Query: 499
Remaining Time: 749.45 mins
---
ID: 241 | Attack Unsuccessful!| GT: 1 | Init. Pred: 1 | Init. Conf: 1.00 | Last Pred: 1 | Last Conf: 1.00 | Query: 499
Remaining Time: 752.80 mins
---
ID: 2

...

ID: 9726 | Attack Unsuccessful!| GT: 9 | Init. Pred: 9 | Init. Conf: 1.00 | Last Pred: 9 | Last Conf: 1.00 | Query: 499
Remaining Time: 20.25 mins
---
ID: 9727 | Wrong Prediction! | GT: 6 | Pred: 8 | Conf: 0.99
Remaining Time: 20.18 mins
---
ID: 9728 | Attack Unsuccessful!| GT: 2 | Init. Pred: 2 | Init. Conf: 1.00 | Last Pred: 2 | Last Conf: 1.00 | Query: 499
Remaining Time: 20.11 mins
---
ID: 9729 | Attack Successful!| GT: 4 | Init. Pred: 4 | Init. Conf: 0.62 | Last Pred: 6 | Last Conf: 0.58 | Query: 5
Remaining Time: 20.03 mins
---
ID: 9730 | Wrong Prediction! | GT: 6 | Pred: 3 | Conf: 1.00
Remaining Time: 19.95 mins
---
ID: 9731 | Attack Successful!| GT: 2 | Init. Pred: 2 | Init. Conf: 1.00 | Last Pred: 6 | Last Conf: 0.45 | Query: 85
Remaining Time: 19.88 mins
---
ID: 9732 | Attack Unsuccessful!| GT: 9 | Init. Pred: 9 | Init. Conf: 1.00 | Last Pred: 9 | Last Conf: 1.00 | Query: 499
Remaining Time: 19.80 mins
---
ID: 9733 | Attack Successful!| GT: 7 | Init. Pred: 7 | Init. Conf: 1.0

ID: 9786 | Attack Successful!| GT: 6 | Init. Pred: 6 | Init. Conf: 1.00 | Last Pred: 1 | Last Conf: 0.47 | Query: 79
Remaining Time: 15.76 mins
---
ID: 9787 | Wrong Prediction! | GT: 8 | Pred: 0 | Conf: 0.74
Remaining Time: 15.69 mins
---
ID: 9788 | Attack Unsuccessful!| GT: 1 | Init. Pred: 1 | Init. Conf: 1.00 | Last Pred: 1 | Last Conf: 1.00 | Query: 499
Remaining Time: 15.61 mins
---
ID: 9789 | Attack Successful!| GT: 6 | Init. Pred: 6 | Init. Conf: 1.00 | Last Pred: 5 | Last Conf: 0.53 | Query: 97
Remaining Time: 15.54 mins
---
ID: 9790 | Attack Successful!| GT: 8 | Init. Pred: 8 | Init. Conf: 1.00 | Last Pred: 2 | Last Conf: 0.51 | Query: 102
Remaining Time: 15.46 mins
---
ID: 9791 | Wrong Prediction! | GT: 9 | Pred: 2 | Conf: 0.83
Remaining Time: 15.39 mins
---
ID: 9792 | Wrong Prediction! | GT: 8 | Pred: 3 | Conf: 0.88
Remaining Time: 15.31 mins
---
ID: 9793 | Wrong Prediction! | GT: 2 | Pred: 5 | Conf: 1.00
Remaining Time: 15.24 mins
---
ID: 9794 | Wrong Prediction! | GT: 1 | P

ID: 9845 | Attack Successful!| GT: 9 | Init. Pred: 9 | Init. Conf: 1.00 | Last Pred: 5 | Last Conf: 0.50 | Query: 244
Remaining Time: 11.37 mins
---
ID: 9846 | Attack Unsuccessful!| GT: 8 | Init. Pred: 8 | Init. Conf: 1.00 | Last Pred: 8 | Last Conf: 0.99 | Query: 499
Remaining Time: 11.30 mins
---
ID: 9847 | Attack Unsuccessful!| GT: 0 | Init. Pred: 0 | Init. Conf: 1.00 | Last Pred: 0 | Last Conf: 1.00 | Query: 499
Remaining Time: 11.22 mins
---
ID: 9848 | Attack Successful!| GT: 2 | Init. Pred: 2 | Init. Conf: 1.00 | Last Pred: 7 | Last Conf: 0.50 | Query: 408
Remaining Time: 11.15 mins
---
ID: 9849 | Wrong Prediction! | GT: 0 | Pred: 2 | Conf: 1.00
Remaining Time: 11.08 mins
---
ID: 9850 | Wrong Prediction! | GT: 7 | Pred: 4 | Conf: 1.00
Remaining Time: 11.00 mins
---
ID: 9851 | Attack Unsuccessful!| GT: 6 | Init. Pred: 6 | Init. Conf: 1.00 | Last Pred: 6 | Last Conf: 1.00 | Query: 499
Remaining Time: 10.93 mins
---
ID: 9852 | Attack Successful!| GT: 0 | Init. Pred: 0 | Init. Conf: 

ID: 9906 | Attack Unsuccessful!| GT: 1 | Init. Pred: 1 | Init. Conf: 1.00 | Last Pred: 1 | Last Conf: 1.00 | Query: 499
Remaining Time: 6.85 mins
---
ID: 9907 | Attack Successful!| GT: 1 | Init. Pred: 1 | Init. Conf: 1.00 | Last Pred: 8 | Last Conf: 0.51 | Query: 102
Remaining Time: 6.78 mins
---
ID: 9908 | Attack Unsuccessful!| GT: 8 | Init. Pred: 8 | Init. Conf: 1.00 | Last Pred: 8 | Last Conf: 1.00 | Query: 499
Remaining Time: 6.70 mins
---
ID: 9909 | Attack Successful!| GT: 1 | Init. Pred: 1 | Init. Conf: 1.00 | Last Pred: 8 | Last Conf: 0.52 | Query: 169
Remaining Time: 6.63 mins
---
ID: 9910 | Wrong Prediction! | GT: 4 | Pred: 7 | Conf: 0.95
Remaining Time: 6.56 mins
---
ID: 9911 | Attack Successful!| GT: 4 | Init. Pred: 4 | Init. Conf: 1.00 | Last Pred: 2 | Last Conf: 0.50 | Query: 195
Remaining Time: 6.48 mins
---
ID: 9912 | Attack Unsuccessful!| GT: 0 | Init. Pred: 0 | Init. Conf: 1.00 | Last Pred: 0 | Last Conf: 1.00 | Query: 499
Remaining Time: 6.41 mins
---
ID: 9913 | Attac

ID: 9965 | Attack Successful!| GT: 2 | Init. Pred: 2 | Init. Conf: 1.00 | Last Pred: 5 | Last Conf: 0.52 | Query: 98
Remaining Time: 2.50 mins
---
ID: 9966 | Attack Successful!| GT: 6 | Init. Pred: 6 | Init. Conf: 1.00 | Last Pred: 2 | Last Conf: 0.51 | Query: 354
Remaining Time: 2.43 mins
---
ID: 9967 | Attack Successful!| GT: 3 | Init. Pred: 3 | Init. Conf: 1.00 | Last Pred: 4 | Last Conf: 0.56 | Query: 70
Remaining Time: 2.35 mins
---
ID: 9968 | Wrong Prediction! | GT: 3 | Pred: 5 | Conf: 1.00
Remaining Time: 2.28 mins
---
ID: 9969 | Attack Successful!| GT: 6 | Init. Pred: 6 | Init. Conf: 0.93 | Last Pred: 9 | Last Conf: 0.57 | Query: 31
Remaining Time: 2.21 mins
---
ID: 9970 | Attack Unsuccessful!| GT: 2 | Init. Pred: 2 | Init. Conf: 1.00 | Last Pred: 2 | Last Conf: 1.00 | Query: 499
Remaining Time: 2.13 mins
---
ID: 9971 | Attack Unsuccessful!| GT: 9 | Init. Pred: 9 | Init. Conf: 1.00 | Last Pred: 9 | Last Conf: 1.00 | Query: 499
Remaining Time: 2.06 mins
---
ID: 9972 | Attack Suc